In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
import json
import numpy as np
import networkx as nx
import pylab as plt

eventsEuropean_Championship = pd.read_json('/content/drive/My Drive/bachelor data/events/events_European_Championship.json')
matchesEuropean_Championship = pd.read_json("/content/drive/My Drive/bachelor data/matches/matches_European_Championship.json")
tagsName = pd.read_csv("/content/drive/My Drive/bachelor data/tags2name.csv")
with open('/content/drive/My Drive/bachelor data/events/events_European_Championship.json') as f:
    d = json.load(f)
tags = json_normalize(data=d, record_path='tags', meta=['id'], record_prefix='_')
teams = pd.read_json("/content/drive/My Drive/bachelor data/teams.json")
eventsEuropean_ChampionshipTags = pd.merge(eventsEuropean_Championship,
                       tags,
                       on='id')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]


In [ ]:
def checkZones2(dataframe, columnName):
  resultSeries = pd.Series([])
  i = -1
  for index, row in dataframe.iterrows():
    x = (row[columnName])['x']
    y = (row[columnName])['y']
    index = i + 1
    resultSeries = resultSeries.append(pd.Series([int(((x // 33.3) * 5) + 1 + (y // 20))]), ignore_index=True)
  return resultSeries

In [ ]:
# Filling 2D matrix of Barcelona team with corresponding passes
def get2dMatrix(passesDataframe, senderColumnName, RecipientColumnName):
  arr = np.arange(225).reshape(15,15)
  i = 0
  j = 0
  passCount = 0
  for i in range(15):
    for j in range(15):
      passCount = 0
      for index, row in passesDataframe.iterrows():
        if(row[senderColumnName] == i+1 and  row[RecipientColumnName] == j+1):
          passCount += 1
        arr[i][j] = passCount    
      j += 1
    i += 1 
  return arr

In [ ]:
def getTeamMatrix(teamId, teamMatchId):
  
  # Getting Successful passes of specific team in specific match
  successfulPasses = eventsEuropean_ChampionshipTags[(eventsEuropean_ChampionshipTags['teamId'] == teamId) & (eventsEuropean_ChampionshipTags['matchId'] == teamMatchId) & (eventsEuropean_ChampionshipTags['eventId'] == 8) & (eventsEuropean_ChampionshipTags['_id'] == 1801)]
  successfulPasses.reset_index(level=0, inplace=True)

  # Adding Sender/Recipient positions
  successfulPasses2 = successfulPasses['positions'].sum()
  del successfulPasses2[1::2]
  x2 = pd.Series(successfulPasses2)
  successfulPassesSender = pd.concat([successfulPasses, pd.DataFrame(x2)], axis=1)
  successfulPassesSender.rename(columns = {0:'Sender'}, inplace = True)

  successfulPasses1 = successfulPassesSender['positions'].sum()
  del successfulPasses1[0::2]
  x = pd.Series(successfulPasses1)
  successfulPassesSenderRecipient = pd.concat([successfulPassesSender, pd.DataFrame(x)], axis=1)
  successfulPassesSenderRecipient.rename(columns = {0:'Recepient'}, inplace = True) 

  # Adding Sender/Recipient zones

  zoneSeriesSender = checkZones2(successfulPassesSenderRecipient, 'Sender')
  zoneDataframeSender = pd.DataFrame(zoneSeriesSender)
  zoneDataframeSender.rename(columns = {0:'SenderZone'}, inplace = True)

  zoneSeriesRecipient = checkZones2(successfulPassesSenderRecipient, 'Recepient')
  zoneDataframeRecipient = pd.DataFrame(zoneSeriesRecipient)
  zoneDataframeRecipient.rename(columns = {0:'RecipientZone'}, inplace = True)

  successfulPassesSenderRecipientZ = pd.concat([successfulPassesSenderRecipient, zoneDataframeSender], axis=1)
  successfulPassesSenderRecipientZone = pd.concat([successfulPassesSenderRecipientZ, zoneDataframeRecipient], axis=1)
  
  # returning the 2D passes Matrix
  return get2dMatrix(successfulPassesSenderRecipientZone, 'SenderZone', 'RecipientZone').flatten()

In [ ]:
def getTeamMatches(teamId):
  teamMatchesIds = np.array([])
  eventsEuropean_ChampionshipTeam = eventsEuropean_Championship[eventsEuropean_Championship['teamId'] == teamId]
  teamMatchesIds = eventsEuropean_ChampionshipTeam.matchId.unique()
  return teamMatchesIds

In [ ]:
def getTeamMatrices(teamId):
  teamMatches = getTeamMatches(teamId)
  teamMatrices = np.zeros((len(teamMatches), 225))
  for i in range(len(teamMatches)):
    teamMatrices[i] = getTeamMatrix(teamId, teamMatches[i])
  return teamMatrices

In [ ]:
franceMatrices = np.zeros((38, 225))
franceMatrices = getTeamMatrices(4418)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
franceMatrices

array([[0., 1., 1., ..., 3., 7., 6.],
       [0., 0., 0., ..., 2., 3., 2.],
       [0., 1., 0., ..., 1., 5., 9.],
       ...,
       [1., 5., 1., ..., 1., 4., 7.],
       [2., 2., 0., ..., 1., 2., 5.],
       [1., 7., 1., ..., 3., 5., 4.]])

In [ ]:
englandMatrices = np.zeros((38, 225))
englandMatrices = getTeamMatrices(2413)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
englandMatrices

array([[ 4.,  4.,  2.,  0.,  0.,  9.,  3.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  2.,  4.,  4.,  3.,  0.,  6.,  4.,  2.,  1.,  1.,  0.,
         0.,  0.,  0.,  0.,  6.,  2.,  5.,  2.,  5.,  0.,  4.,  2.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  1.,  6.,  0.,  1.,  2.,
         2.,  5.,  6.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  5.,  2.,
         0.,  0.,  2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,  1.,  0.,
         0.,  0., 14., 14.,  1.,  1.,  2.,  8.,  0.,  2.,  0.,  1.,  1.,
         4.,  3.,  0.,  0.,  9.,  9.,  6.,  4.,  1.,  1.,  2.,  4.,  0.,
         3.,  0.,  0.,  2.,  2.,  0.,  5.,  3., 11.,  5.,  0.,  2.,  1.,
         4.,  1.,  3.,  0.,  0.,  0.,  1.,  0.,  0.,  2.,  5.,  5.,  5.,
         0.,  0.,  3.,  3.,  2.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  2.,
         3.,  2.,  1.,  0.,  1.,  2.,  4.,  0.,  0.,  0.,  0.,  0.,  3.,
         2.,  0.,  0.,  0.,  4.,  1.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  1.,  1.,  2.,  1.

In [ ]:
czechMatrices = np.zeros((38, 225))
czechMatrices = getTeamMatrices(11555)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
czechMatrices

array([[ 0.,  1.,  0.,  0.,  0.,  2.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  1.,  4.,  0.,  0.,  0.,  4.,  2.,  1.,  2.,  1.,  0.,
         0.,  1.,  2.,  1.,  0.,  2.,  1.,  3.,  0.,  2.,  1.,  2.,  2.,
         3.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  8.,  2.,  3.,  0.,  0.,
         1.,  2.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  1.,  2.,
         0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  2.,  2.,  1.,
         0.,  0.,  2.,  2.,  0.,  0.,  0.,  0.,  2.,  1.,  0.,  0.,  1.,
         2.,  2.,  0.,  0.,  2.,  1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  1.,  0.,  3.,  0.,  2.,  3.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  2.,  0.,  4.,
         0.,  2.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,
         1.,  1.,  0.,  0.,  1.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  1.,  0.,  0., 11.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  4.,  2.,  1.

In [ ]:
icelandMatrices = np.zeros((38, 225))
icelandMatrices = getTeamMatrices(7839)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
icelandMatrices

array([[ 2.,  1.,  0., ...,  1.,  2.,  2.],
       [ 1.,  3.,  1., ...,  2.,  0.,  2.],
       [ 1.,  1.,  1., ...,  0.,  1.,  1.],
       [ 7.,  0.,  0., ...,  2.,  1.,  0.],
       [12.,  7.,  2., ...,  2.,  3.,  8.]])

In [ ]:
austriaMatrices = np.zeros((38, 225))
austriaMatrices = getTeamMatrices(9109)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
austriaMatrices

array([[ 2.,  4.,  3.,  0.,  0.,  4.,  6.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  7.,  7.,  3.,  4.,  0.,  7.,  1.,  0.,  1.,  0.,  1.,
         0.,  0.,  0.,  0.,  4.,  2.,  0.,  2.,  3.,  2.,  6.,  2.,  2.,
         1.,  0.,  0.,  1.,  0.,  0.,  2.,  2.,  1.,  1.,  1.,  3.,  0.,
         3.,  0.,  0.,  0.,  1.,  0.,  2.,  0.,  0.,  0.,  3.,  3.,  2.,
         1.,  0.,  0.,  1.,  3.,  0.,  0.,  0.,  0.,  0.,  1.,  4.,  0.,
         0.,  0.,  8.,  6.,  6.,  0.,  1.,  6.,  1.,  0.,  0.,  0.,  2.,
         1.,  0.,  0.,  0.,  5.,  6.,  5.,  3.,  0.,  2.,  1.,  2.,  1.,
         0.,  1.,  0.,  0.,  0.,  0.,  3.,  4.,  4.,  5.,  7.,  0.,  0.,
         2.,  0.,  5.,  0.,  0.,  1.,  1.,  0.,  0.,  3.,  4.,  3.,  3.,
         1.,  1.,  3.,  3.,  5.,  0.,  0.,  0.,  0.,  1.,  0.,  2.,  2.,
         3.,  8.,  0.,  0.,  1.,  0.,  6.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  1.,  1.,  0., 11.,  3.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  2.,  2.,  3.

In [ ]:
irelandMatrices = np.zeros((38, 225))
irelandMatrices = getTeamMatrices(8274)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
irelandMatrices

array([[ 1.,  2.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  4.,  2.,  0.,  1.,  2.,  7.,  0.,  0.,  1.,  1.,
         0.,  1.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,
         2.,  0.,  0.,  1.,  2.,  0.,  0.,  3.,  2.,  3.,  0.,  2.,  1.,
         1.,  3.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  2.,  1.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  1.,  1.,
         0.,  0.,  2.,  4.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,
         6.,  1.,  1.,  0.,  2.,  3.,  2.,  2.,  1.,  2.,  1.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  2.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  3.,  0.,  0.,  1.,  0.,  2.,  4.,
         0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,
         2.,  4.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  1.,
         2.,  0.,  0.,  0.,  6.,  2.,  1.,  1.,  2.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  2.,  0.,  1.,  1.,  1.

In [ ]:
portugalMatrices = np.zeros((38, 225))
portugalMatrices = getTeamMatrices(9905)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
portugalMatrices

array([[ 3.,  4.,  4., ...,  7.,  6., 14.],
       [ 2.,  4.,  0., ...,  4.,  3.,  6.],
       [ 0.,  0.,  1., ...,  6.,  4.,  8.],
       ...,
       [ 3.,  2.,  0., ...,  1.,  6., 11.],
       [ 6.,  5.,  0., ...,  1.,  3.,  5.],
       [ 6.,  4.,  3., ...,  1.,  1.,  7.]])

In [ ]:
spainMatrices = np.zeros((38, 225))
spainMatrices = getTeamMatrices(1598)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
spainMatrices

array([[ 2.,  1.,  1.,  1.,  0.,  1.,  2.,  2.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  2.,  6.,  2.,  0.,  0.,  3.,  5.,  0.,  2.,  1.,  0.,
         0.,  0.,  0.,  0.,  1.,  3.,  2.,  5.,  1.,  0.,  4.,  0.,  0.,
         3.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  2.,  3.,  2.,  0.,  5.,
         0.,  5.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  4.,  4.,
         0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,
         0.,  0., 14., 16.,  3.,  3.,  1., 11.,  3.,  0.,  1.,  0.,  0.,
         1.,  2.,  0.,  0., 14., 22., 14., 15.,  1., 11., 13.,  1.,  5.,
         1.,  0.,  1.,  0.,  0.,  0.,  4., 16.,  8., 16.,  1.,  3.,  5.,
         3.,  4.,  0.,  0.,  0.,  1.,  0.,  0.,  1., 12., 20., 18.,  6.,
         2.,  2.,  3.,  8., 10.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  2.,
        10.,  6.,  0.,  0.,  0.,  0.,  6.,  0.,  0.,  0.,  0.,  0.,  6.,
        10.,  0.,  0.,  0., 29., 14.,  5.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  6.,  2.,  0.,  0., 17., 12.,  9.

In [ ]:
switzerlandMatrices = np.zeros((38, 225))
switzerlandMatrices = getTeamMatrices(6697)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
switzerlandMatrices

array([[ 4.,  5.,  5.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  7.,  2.,  7.,  0.,  1.,  4.,  4.,  3.,  2.,  3.,  1.,
         1.,  0.,  0.,  0.,  2.,  3.,  1.,  7.,  1.,  2.,  4.,  2.,  3.,
         1.,  1.,  0.,  0.,  0.,  0.,  0.,  9.,  2.,  2.,  1.,  1.,  3.,
         2.,  4.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  2.,  2.,
         0.,  1.,  0.,  2.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  3.,  0.,
         0.,  0., 10., 16.,  2.,  3.,  0.,  4.,  2.,  1.,  0.,  1.,  0.,
         2.,  0.,  1.,  0.,  6., 20., 17., 12.,  1.,  5.,  7.,  2.,  3.,
         0.,  0.,  0.,  0.,  3.,  0.,  8., 14.,  7., 11.,  2.,  2.,  3.,
         1.,  4.,  2.,  0.,  0.,  2.,  3.,  1.,  2., 11., 20., 14.,  8.,
         1.,  2.,  1.,  5.,  3.,  0.,  0.,  0.,  3.,  2.,  0.,  2.,  2.,
         8.,  6.,  0.,  0.,  1.,  1.,  3.,  0.,  0.,  0.,  0.,  0.,  8.,
         1.,  0.,  0.,  0.,  6.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.,  0.,  0.,  0.,  3.,  4.,  4.

In [ ]:
italyMatrices = np.zeros((38, 225))
italyMatrices = getTeamMatrices(3757)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
italyMatrices

array([[ 4.,  2.,  3., ...,  3.,  2., 10.],
       [ 3.,  2.,  2., ...,  2.,  1.,  0.],
       [ 1.,  3.,  3., ...,  1.,  2.,  2.],
       [ 6.,  2.,  3., ...,  2.,  3.,  2.],
       [ 2.,  5.,  5., ...,  1.,  1.,  2.]])

In [ ]:
belgiumMatrices = np.zeros((38, 225))
belgiumMatrices = getTeamMatrices(5629)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
belgiumMatrices

array([[ 2.,  3.,  0., ...,  4.,  5., 12.],
       [ 1.,  1.,  0., ...,  4.,  8., 14.],
       [ 0.,  0.,  0., ...,  0.,  5.,  3.],
       [ 1.,  1.,  0., ...,  2.,  3., 10.],
       [ 4.,  4.,  1., ...,  7., 10., 15.]])

In [ ]:
walesMatrices = np.zeros((38, 225))
walesMatrices = getTeamMatrices(10682)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
walesMatrices

array([[ 0.,  2.,  3., ...,  2.,  4.,  8.],
       [ 2.,  1.,  1., ...,  0.,  0.,  2.],
       [ 3.,  2.,  1., ...,  2.,  6.,  5.],
       [ 2.,  1.,  3., ...,  1.,  2.,  8.],
       [ 5.,  6.,  1., ...,  1.,  6., 14.],
       [ 1.,  2.,  4., ...,  1.,  3.,  3.]])

In [ ]:
romaniaMatrices = np.zeros((38, 225))
romaniaMatrices = getTeamMatrices(11944)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
romaniaMatrices

array([[ 1.,  2.,  0.,  0.,  0.,  5.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  2.,  0.,  2.,  0.,  0.,  4.,  1.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  3.,  1.,  4.,  2.,  2.,  2.,  1.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
         1.,  2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,
         0.,  0.,  0.,  3.,  1.,  0.,  0.,  0.,  0.,  0.,  2.,  1.,  1.,
         0.,  0., 16., 11.,  0.,  0.,  0., 14.,  5.,  0.,  0.,  0.,  1.,
         1.,  1.,  0.,  0.,  6.,  7.,  0.,  0.,  2.,  4.,  2.,  1.,  0.,
         1.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  2.,  2.,  0.,  2.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  5.,  1.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,
         5.,  7.,  0.,  1.,  0.,  2.,  2.,  0.,  0.,  0.,  0.,  0.,  4.,
         0.,  0.,  0.,  0., 10.,  2.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.,  1.,  0.,  0.,  3.,  1.,  0.

In [ ]:
albaniaMatrices = np.zeros((38, 225))
albaniaMatrices = getTeamMatrices(8731)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
albaniaMatrices

array([[ 4.,  1.,  0.,  0.,  0.,  1.,  2.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  4.,  8.,  1.,  0.,  2.,  2.,  0.,  1.,  1.,  1.,
         0.,  1.,  0.,  0.,  2.,  6.,  7.,  5.,  1.,  4.,  2.,  4.,  1.,
         5.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  7.,  9.,  9.,  1.,  1.,
         2.,  2.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,  4.,  9.,
         0.,  0.,  0.,  3.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,  0.,
         0.,  0.,  5.,  6.,  1.,  0.,  0.,  7.,  1.,  2.,  0.,  0.,  0.,
         1.,  0.,  0.,  0., 12.,  8.,  7.,  1.,  0.,  3.,  1.,  2.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  5.,  6., 14.,  6.,  2.,  0.,  0.,
         0.,  0.,  1.,  0.,  1.,  1.,  2.,  2.,  0.,  3.,  2., 10.,  6.,
         2.,  1.,  3.,  2.,  3.,  0.,  0.,  2.,  3.,  0.,  0.,  0.,  0.,
         7., 11.,  0.,  0.,  1.,  1.,  5.,  0.,  0.,  0.,  0.,  0.,  2.,
         1.,  0.,  0.,  0., 13.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  3.,  2.

In [ ]:
germanyMatrices = np.zeros((38, 225))
germanyMatrices = getTeamMatrices(3148)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
germanyMatrices

array([[ 1.,  2.,  1., ...,  5.,  6., 12.],
       [ 1.,  7.,  0., ...,  1.,  6.,  1.],
       [ 7.,  3.,  3., ...,  5., 10., 11.],
       [ 4.,  4.,  0., ...,  3., 13., 16.],
       [ 4.,  7.,  3., ...,  5., 22., 36.],
       [ 0.,  8.,  1., ...,  6., 11., 16.]])

In [ ]:
polandMatrices = np.zeros((38, 225))
polandMatrices = getTeamMatrices(13869)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
polandMatrices

array([[ 0.,  3.,  0., ...,  2.,  4., 21.],
       [ 0.,  1.,  1., ...,  1.,  2.,  4.],
       [ 3.,  1.,  0., ...,  2.,  1.,  8.],
       [ 1.,  2.,  2., ...,  1.,  1.,  2.],
       [ 7.,  5.,  2., ...,  2.,  3., 16.]])

In [ ]:
russiaMatrices = np.zeros((38, 225))
russiaMatrices = getTeamMatrices(14358)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
russiaMatrices

array([[ 3.,  2.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  2.,  2.,  4.,  9.,  1.,  3.,  0.,  2.,  5.,  0.,  0.,
         0.,  0.,  1.,  1.,  1.,  4.,  3.,  3.,  0.,  2.,  0.,  4.,  2.,
         1.,  1.,  0.,  1.,  0.,  0.,  0.,  3.,  1.,  0.,  1.,  1.,  3.,
         2.,  0.,  5.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  3.,  1.,  3.,
         0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  4.,  1.,
         0.,  0.,  9.,  9.,  2.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,
         2.,  0.,  1.,  0.,  9.,  5.,  3.,  9.,  1.,  3.,  2.,  1.,  0.,
         2.,  0.,  2.,  0.,  1.,  0.,  5.,  6.,  6.,  5.,  3.,  1.,  1.,
         1.,  1.,  0.,  0.,  0.,  3.,  0.,  1.,  0.,  6.,  3., 11., 10.,
         1.,  1.,  4.,  3.,  6.,  0.,  0.,  1.,  2.,  1.,  0.,  1.,  3.,
         9.,  9.,  0.,  0.,  1.,  2.,  2.,  0.,  0.,  0.,  0.,  0.,  1.,
         2.,  0.,  0.,  0.,  2.,  3.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  2.,  0.,  0.,  0.,  0.,  1.,  0.,  1.

In [ ]:
slovakiaMatrices = np.zeros((38, 225))
slovakiaMatrices = getTeamMatrices(14496)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
slovakiaMatrices

array([[ 5.,  5.,  0.,  0.,  0.,  4.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  6.,  7.,  3.,  5.,  0.,  3.,  7.,  2.,  6.,  1.,  1.,
         1.,  0.,  0.,  0.,  2.,  5.,  0.,  2.,  1.,  1.,  2.,  2.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  6.,  0.,  4.,  2.,  0.,  1.,
         6.,  7.,  7.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  6.,  3.,
         1.,  0.,  0.,  3.,  1.,  0.,  0.,  0.,  0.,  1.,  3.,  4.,  2.,
         0.,  0.,  7.,  9.,  1.,  1.,  0.,  6.,  1.,  0.,  0.,  0.,  1.,
         3.,  0.,  2.,  0.,  4., 11.,  3.,  7.,  2.,  2.,  7.,  2.,  2.,
         0.,  1.,  1.,  0.,  3.,  0.,  3.,  6.,  6.,  9.,  1.,  1.,  4.,
         5.,  0.,  1.,  0.,  3.,  0.,  3.,  1.,  2.,  7.,  7.,  8., 10.,
         0.,  0.,  0., 10.,  5.,  0.,  0.,  0.,  5.,  2.,  0.,  1.,  3.,
         7.,  3.,  0.,  0.,  0.,  6.,  3.,  0.,  0.,  0.,  0.,  0.,  3.,
         2.,  0.,  0.,  1.,  3.,  5.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  1.,  1.,  1.,  0.,  0.,  4.,  4.,  3.

In [ ]:
croatiaMatrices = np.zeros((38, 225))
croatiaMatrices = getTeamMatrices(9598)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
croatiaMatrices

array([[ 2.,  1.,  0.,  0.,  1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  3.,  1.,  2.,  1.,  3.,  2.,  1.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  2.,  2.,  2.,  3.,  2.,  3.,  3.,
         1.,  1.,  0.,  1.,  0.,  0.,  0.,  3.,  4.,  3.,  3.,  1.,  2.,
         4.,  0.,  5.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  4.,  2.,
         0.,  0.,  0.,  2.,  5.,  0.,  0.,  0.,  1.,  1.,  2.,  1.,  0.,
         0.,  0.,  5.,  3.,  1.,  1.,  0.,  4.,  2.,  0.,  0.,  0.,  1.,
         2.,  1.,  0.,  1.,  1.,  3.,  2.,  1.,  0.,  1.,  1.,  0.,  0.,
         0.,  0.,  1.,  2.,  0.,  0.,  4.,  1.,  3.,  5.,  2.,  1.,  0.,
         2.,  2.,  2.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  4.,  2.,  8.,
         0.,  0.,  0.,  3.,  8.,  0.,  0.,  2.,  4.,  0.,  0.,  0.,  1.,
         6.,  8.,  0.,  0.,  0.,  3.,  7.,  0.,  0.,  0.,  0.,  0.,  1.,
         1.,  0.,  0.,  0.,  5.,  1.,  4.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  2.

In [ ]:
turkeyMatrices = np.zeros((38, 225))
turkeyMatrices = getTeamMatrices(4687)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
turkeyMatrices

array([[ 3.,  0.,  1.,  1.,  0.,  4.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  3.,  2.,  0.,  1.,  0.,  3.,  2.,  4.,  1.,  0.,  0.,
         1.,  0.,  0.,  0.,  7.,  3.,  5.,  0.,  1.,  2.,  3.,  3.,  2.,
         0.,  0.,  0.,  2.,  0.,  0.,  1.,  2.,  3.,  4.,  4.,  1.,  0.,
         0.,  4.,  2.,  0.,  0.,  0.,  2.,  0.,  0.,  2.,  0.,  2.,  1.,
         0.,  0.,  0.,  2.,  1.,  0.,  0.,  0.,  0.,  0.,  2.,  1.,  0.,
         0.,  0., 14.,  9.,  2.,  0.,  0.,  2.,  2.,  0.,  0.,  1.,  0.,
         2.,  1.,  1.,  0.,  9.,  9.,  6.,  3.,  4.,  3.,  0.,  3.,  0.,
         2.,  1.,  0.,  0.,  1.,  0.,  4.,  7.,  8.,  5.,  4.,  2.,  1.,
         4.,  1.,  0.,  0.,  0.,  0.,  3.,  1.,  2.,  4.,  5.,  9.,  6.,
         0.,  1.,  0.,  2.,  6.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  4.,
         8., 10.,  0.,  0.,  1.,  4.,  4.,  0.,  0.,  0.,  0.,  0.,  1.,
         1.,  1.,  0.,  0.,  3.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  1.,  0.,  0.,  0.,  2.,  0.,  1.

In [ ]:
hungaryMatrices = np.zeros((38, 225))
hungaryMatrices = getTeamMatrices(10451)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
hungaryMatrices

array([[10.,  5.,  5.,  0.,  0.,  2.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  5.,  3.,  2.,  0.,  1.,  5.,  6.,  0.,  1.,  0.,  1.,
         0.,  0.,  1.,  0., 10.,  3.,  3.,  2.,  2.,  2.,  1.,  1.,  1.,
         4.,  1.,  1.,  0.,  0.,  1.,  0.,  1.,  4.,  3.,  4.,  0.,  0.,
         2.,  2.,  2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,  5.,
         0.,  0.,  1.,  4.,  4.,  0.,  0.,  0.,  2.,  0.,  1.,  1.,  0.,
         1.,  0.,  7.,  9.,  1.,  0.,  0.,  3.,  1.,  1.,  0.,  0.,  1.,
         1.,  1.,  0.,  1.,  6.,  6.,  5.,  4.,  0.,  3.,  3.,  0.,  0.,
         0.,  0.,  0.,  1.,  1.,  0.,  2.,  4.,  0.,  2.,  1.,  1.,  1.,
         4.,  0.,  2.,  0.,  0.,  2.,  0.,  0.,  1.,  2.,  1.,  1.,  5.,
         1.,  1.,  0.,  3.,  2.,  0.,  0.,  0.,  2.,  2.,  0.,  0.,  1.,
         4.,  3.,  0.,  0.,  1.,  3.,  4.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  4.,  6.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  4.,  0.,  1.,  0.,  5.,  1.,  0.

In [ ]:
northenIrelandMatrices = np.zeros((38, 225))
northenIrelandMatrices = getTeamMatrices(10962)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
northenIrelandMatrices

array([[ 3.,  6.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  6.,  6.,  5.,  8.,  2.,  4.,  3.,  0.,  1.,  0.,  0.,
         1.,  0.,  0.,  0.,  3.,  7.,  1.,  1.,  1.,  2.,  3.,  1.,  0.,
         1.,  0.,  2.,  1.,  0.,  1.,  0.,  4.,  2.,  3.,  1.,  0.,  1.,
         3.,  2.,  2.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  2.,  0.,
         0.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,
         0.,  0.,  2.,  3.,  1.,  0.,  0.,  8.,  1.,  0.,  0.,  1.,  1.,
         2.,  3.,  0.,  0.,  5.,  3.,  3.,  1.,  0.,  0.,  1.,  0.,  1.,
         0.,  0.,  0.,  0.,  1.,  2.,  1.,  2.,  3.,  2.,  1.,  1.,  1.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,
         0.,  0.,  1.,  1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,
         3.,  0.,  0.,  0.,  8.,  3.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.,  1.,  0.,  0.,  4.,  1.,  0.

In [ ]:
swedenMatrices = np.zeros((38, 225))
swedenMatrices = getTeamMatrices(7047)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
swedenMatrices

array([[ 1.,  2.,  0.,  0.,  0.,  1.,  2.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  1.,  2.,  3.,  1.,  0.,  3.,  5.,  2.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,  5.,  1.,  2.,
         0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  4.,  3.,  0.,  0.,
         2.,  2.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  3.,
         0.,  0.,  0.,  4.,  2.,  0.,  0.,  0.,  0.,  0.,  3.,  0.,  1.,
         0.,  0., 12., 12.,  1.,  1.,  0.,  7.,  0.,  2.,  0.,  0.,  1.,
         1.,  1.,  1.,  0., 15., 10.,  1.,  2.,  2.,  3.,  4.,  2.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.,  4.,  5.,  2.,  5.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  2.,  6.,  3.,  7.,  7.,
         2.,  2.,  1.,  1.,  2.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,
        10., 17.,  0.,  2.,  0.,  3.,  5.,  0.,  0.,  0.,  0.,  0.,  4.,
         1.,  0.,  0.,  0., 15.,  3.,  4.,  3.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  2.,  0.,  0.,  0.,  2.,  6.,  1.

In [ ]:
ukraineMatrices = np.zeros((38, 225))
ukraineMatrices = getTeamMatrices(14622)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
ukraineMatrices

array([[ 4.,  2.,  0.,  0.,  0.,  1.,  2.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  3.,  3.,  4.,  1.,  0.,  4.,  0.,  1.,  1.,  0.,  0.,
         0.,  0.,  1.,  0.,  2.,  5.,  1.,  4.,  1.,  3.,  2.,  1.,  0.,
         0.,  0.,  0.,  0.,  1.,  2.,  0.,  3.,  5.,  6.,  5.,  0.,  1.,
         2.,  2.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  6.,  1.,
         0.,  0.,  0.,  3.,  4.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,
         1.,  0.,  5.,  2.,  2.,  0.,  0.,  2.,  2.,  0.,  0.,  0.,  2.,
         2.,  0.,  0.,  0.,  1.,  0.,  2.,  1.,  1.,  0.,  0.,  1.,  0.,
         1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,
         1.,  0.,  2.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  2.,  2.,  3.,
         0.,  0.,  0.,  1.,  3.,  0.,  1.,  0.,  2.,  4.,  0.,  1.,  0.,
         1.,  9.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  2.,
         1.,  0.,  0.,  0.,  5.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  1.,  3.,  1.,  0.,  0.,  3.,  2.,  1.

In [ ]:
from google.colab import files
import pickle
with open('eu.pickle', 'wb') as f:
  pickle.dump([franceMatrices, englandMatrices, czechMatrices, icelandMatrices, austriaMatrices, northenIrelandMatrices, portugalMatrices, spainMatrices, switzerlandMatrices, italyMatrices, belgiumMatrices, walesMatrices, romaniaMatrices, albaniaMatrices, germanyMatrices, polandMatrices, russiaMatrices, slovakiaMatrices, croatiaMatrices, turkeyMatrices, hungaryMatrices, irelandMatrices, swedenMatrices, ukraineMatrices], f)
files.download('eu.pickle')  